# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "qwen3:8b"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

In [ ]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [6]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

<think>
Okay, the user wants to know about the business applications of Generative AI. Let me start by recalling what Generative AI is. It's AI that creates new content, like text, images, or code. Now, how does that apply to businesses?

First, customer service comes to mind. Chatbots and virtual assistants can handle a lot of queries. But wait, there's more. Maybe they can generate personalized responses. Oh right, like using customer data to tailor replies. That's a good point.

Then there's content creation. Companies need a lot of content for marketing, social media, blogs. Generative AI can help write articles, create social media posts, even generate video scripts. That's a big one. But I should mention specific examples like marketing copy or product descriptions.

Product design is another area. For example, in fashion or automotive, AI can generate designs or prototypes. Maybe even simulate how a product would look or function. That could save time and costs.

Marketing and a

## Alternative approach - using OpenAI python library to connect to Ollama

In [7]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

<think>
Okay, I need to describe some business applications of Generative AI. Let me start by recalling what generative AI is. It's AI that can create new content, like text, images, music, etc., based on the data it's trained on. So, how is this used in business? Let me brainstorm.

First, customer service. Companies use chatbots, right? Generative AI can create natural language responses, making chatbots more human-like. That improves customer experience.

Then there's content creation. For example, generating marketing copy, social media posts, or product descriptions. This could save time for marketing teams. Maybe even creating personalized content for different customers?

Product development might be another area. Generative AI could help design products by creating new product concepts or even prototypes. Like in fashion or automotive industries, generating design ideas.

In finance, maybe generating reports or analyzing data to predict market trends. Also, fraud detection coul

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [ ]:
!ollama pull deepseek-r1:1.5b

In [ ]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [11]:
# Step 1: Create your prompts
from openai import OpenAI
from IPython.display import Markdown, display
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
system_prompt = "Jesteś asystentem do pisania maili akademkich. Pomagasz studentom pisać maile do prowądzących. Sprawdź wstępną wersję pod kątem gramatyki, ortografii, poprawności oraz odpowiednio szacunku w stronę prowadzących. Przepisz maila zgodnie z tym jak uważasz powinno być oraz dopisz ewentualne krótkie sugestie gdy nie jesteś pewny."
user_prompt = """
    Treść maila: \n\n
"""
user_prompt += """
Szanowany Panie Doktorze,
chciałbym się dowiedzieć jakie były finalnie przeliczniki poszczególnych części przedmiotu z Przetwarzania Rozproszonego oraz od ilu % była ocena 3.5 bo nie znalazłem tych informacji na enauczaniu. Z tego co wiem ktoś pisał do Pana by ustalić przelicznik 40-40-20 i wtedy wychodzi mi więcej niż 60% (56% z egzaminu, 65% z labów oraz zrobilIśmy w pełni 2 części projektu) a mimo to dostałem ocenę 3.0.
Z wyrazami szacunku
Kornel Wo. 198349, inf. 4 sem. gr. 7A
"""
user_prompt += "Tytuł: Ocena z Przetwarzania Rozproszonego"

# Step 2: Make the messages list

messages = [{"role": "system", "content": system_prompt}, {"role":"user","content": user_prompt}] # fill this in

# Step 3: Call OpenAI

response = openai.chat.completions.create(
        model = MODEL,
        messages = messages
    )

display(Markdown(response.choices[0].message.content))

<think>
Okay, let me look at this email. The user is a student who needs to ask their lecturer about grading details for a course called Przetwarzania Rozproszonego. The main points are: they want to know the final conversion ratios for each part of the course and how the 3.5 grade was calculated because they couldn't find this info on the course website. They mention someone else asked about the 40-40-20 split and their own calculations don't match their received grade of 3.0.

First, I need to check for grammar, spelling, and proper respect. The original starts with "Szanowany Panie Doktorze," which is correct. But maybe "Szanowny Panie Doktorze" is more formal. The body seems okay but could be more structured and polite.

The user wrote "chciałbym się dowiedzieć" – that's correct. However, the phrase "bo nie znalazłem tych informacji na enauczaniu" has a typo: "enauczaniu" should be "enauczaniu" (corrected). Also, "zrobilIśmy" has a typo with the capital I, should be "zrobiliśmy." 

The structure could be improved. Starting with a subject line, then a clear request. The current email is a bit abrupt. Maybe separate the request into paragraphs for clarity. Also, the user mentions someone else asked about the 40-40-20 split. Including that might add context but could be better phrased to show they're not just asking randomly.

I should make sure the tone is respectful and polite. The original uses "Szanowny Panie Doktorze" which is appropriate, but maybe adding "z wyrazami szacunku" at the end is okay, though sometimes it's placed after the signature. Also, the user's ID and group are at the end, which is acceptable.

In the response, I'll correct the typos, restructure the email for clarity, and ensure the tone is appropriate. The user should mention their student ID and group, which they did. The suggestion to add the course title in the subject line is good. Also, making sure the explanation of their calculations is clear without sounding confrontational.

So, the revised email will have a proper subject line, correct grammar, structured paragraphs, and maintain the respect towards the lecturer. The user should also check the course website again to confirm there's no info there, as the original email mentions that. The final note to the lecturer should be polite and not accusatory.
</think>

**Poprawiona wersja maila (wypunktowana):**

---

**Tytuł:** Wyszukiwanie informacji o przeliczniku i ocenie z Przetwarzania Rozproszonego  

**Treść maila:**  

Szanowny Panie Doktorze,  

Zapraszam do przeczytania poniższego maila. W ramach studiów na kierunku Informatyka (grupa 7A), IV semestr, chciałbym zapytać o możliwość uzyskania szczegółowych informacji dotyczących przelicznika poszczególnych części przedmiotu *Przetwarzanie Rozproszonego* oraz sposobu przeliczenia na ocenę (np. na skali 3.5).  

W trakcie analizy wyników egzaminu i prac laboratoryjnych, dostrzegłem niedokładność w obliczeniach – zgodnie z przypuszczeniem, że przelicznik wynosił 40-40-20, suma moich wyników powinna wynosić więcej niż 60% (np. 56% z egzaminu, 65% z labów, 200% z projektu). Niestety, otrzymana ocena została ustalona na 3.0, co generuje pytania dotyczące metody wyliczenia końcowego wyniku.  

Oczywiście, nie chcę wyrażać wątpliwości względem Pana doktora, ale chciałbym mieć możliwość sprawdzenia przelicznika i zasady oceniania – aby w przyszłości lepiej się przygotowywać. Proszę o pomoc w wskazaniu odpowiednich danych lub podanie ich wprost.  

Z wyrazami szacunku,  
Kornel Wo. 198349, inf. 4 sem. gr. 7A  

---

**Sugestie poprawek (jeśli potrzebne):**  
1. **Uzyskanie wersji z podpisem i datą:** W niektórych przypadkach prowadzący oczekują podpisu i daty w formacie „Kornel Wo. 198349, inf. 4 sem. gr. 7A” – zgodnie z podaną formą.  
2. **Zdrowy szare** – Jeśli chcesz wyrazić wątpliwości, najlepiej je sformułować niekwestionująco, np.: *„Zapraszam do przeczytania poniższego maila, by wyjaśnić pewne wątpliwości dotyczące wyników”*.  
3. **Zwięzłość:** Jeśli chcesz uniknąć nadmiernego długiego tekstu, możesz zapytać prosto: *„Czy mógłby Pan podać przelicznik oraz sposoby przeliczania na ocenę?”*.  

*Uwaga:* Jeśli chcesz, mogę Ci pomóc przygotować także wersję maila z bardziej formalnym tonem, np. jeśli daje to lepsze efekty w kontakcie z prowadzącym.